In [ ]:
# THIS FUNCTION READS IN BORIS FILES, WHICH FOLLOW A FORMAT OF ONE XLSX FILE PER VIDEO, AND HAVE 15 ROWS OF HEADER 
# TEXT. SOME CLEANING AND MUTATING IS DONE, MOSTLY TO GET THE VIDEO NUMBER AND CORRECT TIMES 
def mutateBORISandVID(depID,outloc,prh):
    # GET BORIS DATA
    borisloc = "C:\\01_Work\\1Project Home\\1Research\\GRAYS\\_tag data\\"+depID+"\\BORIS\\"       # path to folder with individual audit files
    borislist = os.listdir(borisloc)                                                               # list those files
    boris = pd.DataFrame()

    #READ IN EACH INDIVIDUAL FILE
    for f in borislist:
        vidname = pd.read_excel((borisloc+f), nrows = 1, header = None)
        data = pd.read_excel((borisloc+f), skiprows=15)
        data['vidname'] = vidname.iloc[0,1]
        boris = pd.concat([boris, data])
    #boris.to_excel(outloc+depID+"-tempcompiledBORIS.xlsx")

    # MUTATE BORIS DATA
    # first check the length of the audit file list against the length of vidDN to 
    # make sure they match since names may have changed
    if len(prh.get("vidDN")) != len(borislist):
        print('Differing number of audit files from prh vid DNs...')
    else:
        print('')

    # GET THE VIDEO NUMBER FROM THE FILE NAME 
    dat = boris['vidname'].apply(lambda x: x.split('(')[1] if x.strip() else x) # remove the first (
    boris['vidnum'] = dat.apply(lambda x: x.split(')')[0]) # remove the secon ) and add to dataframe
    boris['vidnum'] = boris['vidnum'].str[:2] # take the first two numbers (for if a series of video numbers were combined)
    boris['vidnum'] = boris['vidnum'].apply(lambda x: x.lstrip('0') if x.strip() else x)
    # 02.12.2024. Changed to above to try to fix bug resulting if the video name is empty (happens when videos were merged during prh)
    #boris['vidnum']= boris['vidnum'].apply(lambda x: x.lstrip('0')) # remove the leading zeroes if present
    boris['vidnum'] = boris['vidnum'].astype(int)
    dat = prh.get('vidDN')
    dat = pd.DataFrame(dat.flatten())
    dat = dat.rename(columns = {0:'DN'})
    boris['vidDN'] = boris['vidnum'].apply(lambda x: dat['DN'].iloc[x] if x < len(dat) else None)

    # MUTATE VIDEO DATA
    viddat = pd.DataFrame()
    # pull the video name from prh file
    viddat['vidname'] = [item[0][0] for item in prh.get('vidNam')]
    # get the prh video number from the prh video name
    dat = viddat['vidname'].apply(lambda x: x.split('W')[1])
    viddat['vidnum'] = dat.apply(lambda x: x.split('.')[0])
    viddat['vidnum'].astype(int)
    viddat['vidnum']= viddat['vidnum'].apply(lambda x: x.lstrip('0'))
    # get the viddn
    viddat['vidDN'] = [item[0] for item in prh.get('vidDN')]
    # get the video start index
    dat = prh.get('DN')
    dat = dat.flatten()
    for i in range(len(viddat)): 
        searchDN = viddat.loc[viddat.index[i], 'vidDN']
        viddat.at[i,'indStartVid'] = np.where(dat >= searchDN)[0][0]
    
    # FINALIZE EDITS TO BORIS DATA
    # add the start index on the videos to the boris data
    viddat['vidnum'] = viddat['vidnum'].index
    boris = pd.merge(viddat, boris, how='inner', left_on='vidnum', right_on='vidnum')

    boris.drop(['Media file path', 'Total length', 'FPS'], axis = 1, inplace=True)
    boris.rename(columns={'vidname_x': 'prhvidname', 'vidname_y': 'borisvidname', 'vidDN_y':'vidDN'}, inplace=True)

    boris['ObsInd'] = round(boris['indStartVid']+(boris['Time']*fs))
    # get the observation ds
    for i, v in boris['vidDN'].items():
        boris.loc[i, 'vidDS'] = dt.fromordinal(int(v)) + timedelta(days=v%1) - timedelta(days = 366)
        boris.loc[i, 'obsDS'] = boris.loc[i, 'vidDS']+timedelta(seconds = boris.loc[i, 'Time'])

    boris.to_excel(outloc+depID+"-compiledBORIS.xlsx")
    print('Saved compiled BORIS file to directory...')
    return boris, viddat


# THIS FUNCTION READS IN START AND END INDICES OF FEEDING EVENTS AND RETURNS ALL
# BEHAVIORAL OBSERVATIONS MADE IN BORIS OCCURRING DURING THAT PERIOD. SAVES A 
# ROLL+BORIS STRUCTURE TO OUTPUT DIRECTORY
def getfeedBORISObs(feed, boris, outloc, depID):
    feed['feedInd'] = feed.index
    mtchs = pd.DataFrame()
    for i in range(len(feed)): 
        sI = feed.at[i, 'sFEI']
        eI = feed.at[i, 'eFEI']
        fmatchs = pd.DataFrame() #temporary dataframe for storing matches
        fmtchs = boris[boris['ObsInd'].between(sI, eI)] #look for behavioral observations occuring during this feeding event
        fmtchs.drop(['prhvidname', 'vidnum', 'Time', 'Modifier 1', 'Modifier 2', 'Status', 'Unnamed: 8'], axis = 1, inplace=True) #delete unnecessary columns
        fmtchs['feedInd'] = repr(i) #get index of the current feeding event (in feed dataframe)
        fmtchs = fmtchs.groupby('feedInd').agg(list) #aggregate all the observations into lists by column
        fmtchs['feedInd'] = i
        mtchs = pd.concat([mtchs, fmtchs], axis = 0, ignore_index=True)
    feed = pd.merge(feed, mtchs, on='feedInd', how='outer')
    feed.to_excel(outloc+depID+"-roll+BORIS.xlsx")
    print('Added BORIS observations to roll data and exported to output dir...')
    return feed

# THIS FUNCTION GENERATES A FIGURE WITH 3 SUBPLOTS: DEPTH PROFILE, ROLL PROFILE, AND ALL 
# BORIS BEHAVIORAL OBSERVATIONS AS VERTICAL LINES COLORED BY TYPE. ONLY THE FOLLOWING OBSERVATIONS ARE 
# PLOTTED: OBSERVATION OF PREY (BURROWS OR OTHERWISE), FEEDING BEHAVIOR (CREATION OF SEDIMENT PLUMES) 
# CURRENTLY SEEMS TO HAVE AN OFFSET BUG.
def plotBORIS(prhObs, outloc, depID):
    filtPDat = prhObs[prhObs['ObsType'].str.strip() != '']
    print(filtPDat.head())
    obs_type_colors = {'Prey': 'mediumvioletred', 'Social': 'darkorchid', 'Feed': 'seagreen'}
    grouped_data = filtPDat.groupby('ObsType')

    fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(18,12))
    # Observational plot
    for obs_type, group in grouped_data:
        obs_color = obs_type_colors.get(obs_type, 'black')
        ax3.vlines(x=group['DS'], ymin=0, ymax=1, color=obs_color, label=obs_type)
        
    for i, v in enumerate(periodsNOCAM['sDS']):
        sI = periodsNOCAM.loc[i,'sDS']
        eI = periodsNOCAM.loc[i,'eDS']
        ax3.axvspan(sI, eI, color = 'darkgray', alpha = 0.5)
        
    #depth plot
    ax1.plot(prhObs['DS'], prhObs['p'], color = 'steelblue')
    ax1.invert_yaxis()

    # roll plot
    ax2.plot(prhObs['DS'], prhObs['roll'], color = 'firebrick')

    common_xlim = (tagonDS, tagoffDS)  # Replace start_date and end_date with your desired limits
    ax1.set_xlim(common_xlim)
    ax2.set_xlim(common_xlim)
    ax3.set_xlim(common_xlim)
    ax3.set_ylim([0,1])

    #handles, labels = ax3.get_legend_handles_labels()
    #plt.legend(handles, labels, loc='upper left')
    # Shrink current axis's height by 10% on the bottom
    box = ax3.get_position()
    ax3.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])

    # Put a legend below current axis
    ax3.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol = 3)

    ax1.set_xticks([])
    ax1.set_xlabel('')
    ax2.set_xticks([])
    ax2.set_xlabel('')
    ax3.set_yticks([])
    ax3.set_ylabel('')
    ax1.set_ylabel('depth(m)')
    ax2.set_ylabel('roll(rad)')
    fig.suptitle('BORIS Observations v depth, roll: '+depID, fontsize=20)
    plt.savefig(outloc+"\\"+depID+"_BORISObs-depth-roll.png")
    return fig









